## Various attempts at tool calling using `granite-3-8b-instruct`

In [ ]:
#installations
%pip install --quiet langchain
%pip install --quiet --upgrade langchain_ibm
%pip install --quiet langchain_core
%pip install --quiet langchain_community
%pip install --quiet youtube_search
%pip install --quiet wikipedia 
%pip install --quiet langgraph
%pip install --quiet ionic-langchain

In [2]:
#imports
import os
import requests

from wikipedia import summary
from datetime import datetime
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ibm import ChatWatsonx
from langchain_community.tools import YouTubeSearchTool
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_core.tools import Tool
from langchain_community.tools import YouTubeSearchTool
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain_core.output_parsers import PydanticToolsParser
from ionic_langchain.tool import IonicTool

load_dotenv(os.getcwd()+"/.env", override=True)

True

In [3]:
WATSONX_APIKEY = os.getenv('WATSONX_APIKEY', "<YOUR_WATSONX_APIKEY_HERE>")

WATSONX_PROJECT_ID = os.getenv('WATSONX_PROJECT_ID', "<YOUR_WATSONX_PROJECT_ID_HERE>")

URL = "https://us-south.ml.cloud.ibm.com"

OPENWEATHERMAP_API_KEY = os.getenv('OPENWEATHERMAP_API_KEY', "<YOUR_OPEN_WEATHERMAP_API_KEY_HERE>")

In [4]:
llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url = URL,
	apikey = WATSONX_APIKEY,
	project_id = WATSONX_PROJECT_ID,
    params = {
        "decoding_method": "greedy",
        "temperature": 0, 
        "min_new_tokens": 5, 
        "max_new_tokens": 2000
    }
)

In [5]:
weather = OpenWeatherMapAPIWrapper()

weather_search = Tool(
    name="weather_search",
    description="Get weather for a city and country code, e.g. Athens, GR",
    func=weather.run,
)

youtube = YouTubeSearchTool()

youtube_search = Tool(
    name="youtube_search",
    description="Search YouTube for video links.",
    func=youtube.run,
)

ionic_search = IonicTool().tool()

tools = [weather_search, youtube_search, ionic_search]

# Different approaches for tool calling:

### Using `create_react_agent` along with `agent_executor`:

This is the best performing option at the moment.

In [6]:
agent_executor = create_react_agent(llm, tools)

In [7]:
user_query = "What are some YouTube videos about Greece"
response = agent_executor.invoke({"messages": HumanMessage(content=user_query)})
response["messages"]

[HumanMessage(content='What are some YouTube videos about Greece', additional_kwargs={}, response_metadata={}, id='d57f04a2-c1e6-4727-8535-2d9fc81365b6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-f5fe7898cfb747c68bf8565583af1e8a', 'type': 'function', 'function': {'name': 'youtube_search', 'arguments': '{"query": "Greece"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 578, 'total_tokens': 600}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-1253e6edaf6547c8a7f4f37f0721a3dd', tool_calls=[{'name': 'youtube_search', 'args': {'query': 'Greece'}, 'id': 'chatcmpl-tool-f5fe7898cfb747c68bf8565583af1e8a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 578, 'output_tokens': 22, 'total_tokens': 600}),
 ToolMessage(content="['https://www.youtube.com/watch?v=NMlBB2pK5qo&pp=ygUGR3JlZWNl', 'https://www.youtube.com/watch?v=j6GQjc3mFQk&pp=ygUGR3JlZWNl']", name

In [8]:
user_query = "What is the weather in Athens, GR"
response = agent_executor.invoke({"messages": HumanMessage(content=user_query)})
response["messages"]

[HumanMessage(content='What is the weather in Athens, GR', additional_kwargs={}, response_metadata={}, id='16548304-80fb-4363-8a57-d3bc342adbb0'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-224ad83053954394bd53bdb0b166043a', 'type': 'function', 'function': {'name': 'weather_search', 'arguments': '{"__arg1": "Athens, GR"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 579, 'total_tokens': 605}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-2dffa5f3c9c3443ea36ae7a5961c10f1', tool_calls=[{'name': 'weather_search', 'args': {'__arg1': 'Athens, GR'}, 'id': 'chatcmpl-tool-224ad83053954394bd53bdb0b166043a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 579, 'output_tokens': 26, 'total_tokens': 605}),
 ToolMessage(content='In Athens, GR, the current weather is as follows:\nDetailed status: few clouds\nWind speed: 6.69 m/s, direction: 340°\nHumidity: 60

In [9]:
user_query = "Find some suitcases for less than $150"
response = agent_executor.invoke({"messages": HumanMessage(content=user_query)})
response["messages"]

content='[{"products": [{"links": [{"text": "Details", "type": "pdp", "url": "https://go.ionic.click/4ggDYD"}], "merchant_name": "Adorama", "merchant_product_id": "product_9102_SLAHLKY", "name": "Slinger - Slinger Leather Case for AirTag Finder Compatible with AirTags 2021 (Yellow)", "price": "$9.95", "status": "available", "thumbnail": "https://www.adorama.com/images/xlarge/SLAHLKY.JPG", "brand_name": "Slinger", "upc": "192223165557"}, {"links": [{"text": "Details", "type": "pdp", "url": "https://go.ionic.click/XF5wTp"}], "merchant_name": "Walmart", "merchant_product_id": "3213314582", "name": "Tripumer 110L Extra Large Moving Bag Travel Luggage Bag Foldable Waterproof Storage Bag Double Zipper and Strong Handle Moving Available Black", "price": "$10.99", "status": "available", "thumbnail": "https://i5.walmartimages.com/asr/1030ce8d-d6f3-441d-af01-aa452b3f277b.526c53998b621552e03c9a7c75ca12fe.jpeg?odnHeight=100&odnWidth=100&odnBg=FFFFFF", "brand_name": "Tripumer", "upc": "639846850243

[HumanMessage(content='Find some suitcases for less than $150', additional_kwargs={}, response_metadata={}, id='58a0e028-e7ec-4637-af09-2cd7dba3bc07'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-7db5539f8c7c47f699169d99fd9f9f6c', 'type': 'function', 'function': {'name': 'ionic_commerce_shopping_tool', 'arguments': '{"__arg1": "suitcases, 4, 150, 1500"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 580, 'total_tokens': 621}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-887d3df7cf9b424e8c666459dd44e57a', tool_calls=[{'name': 'ionic_commerce_shopping_tool', 'args': {'__arg1': 'suitcases, 4, 150, 1500'}, 'id': 'chatcmpl-tool-7db5539f8c7c47f699169d99fd9f9f6c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 580, 'output_tokens': 41, 'total_tokens': 621}),
 ToolMessage(content='[{"products": [{"links": [{"text": "Details", "type": "pdp", "url": "htt

### Another approach is using `bind_tools`:
This returns the function call but does not actually show the output of the function call

In [56]:
llm_with_tools = llm.bind_tools(tools)
query = 'What is the weather in Athens, GR'
llm_with_tools.invoke(query).tool_calls

[{'name': 'weather_search',
  'args': {'__arg1': 'Athens, GR'},
  'id': 'chatcmpl-tool-ff50f21ee32c4615b66b0b12f4e93ce2',
  'type': 'tool_call'}]

#### We can [pass tool calls back to model](https://python.langchain.com/v0.2/docs/how_to/tool_results_pass_to_model/) to get the `AIMessage` we are looking for.

In [54]:
messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
print(ai_msg.tool_calls)
messages.append(ai_msg)


[{'name': 'weather_search', 'args': {'__arg1': 'Athens, GR'}, 'id': 'chatcmpl-tool-51501bdb5f954bed96d9b122b3b0d2b3', 'type': 'tool_call'}]


In [57]:
print(ai_msg)

content='' additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-51501bdb5f954bed96d9b122b3b0d2b3', 'type': 'function', 'function': {'name': 'weather_search', 'arguments': '{"__arg1": "Athens, GR"}'}}]} response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 223, 'total_tokens': 249}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'} id='chat-7b7876f7b3314ae8ad4f9a55360cde3a' tool_calls=[{'name': 'weather_search', 'args': {'__arg1': 'Athens, GR'}, 'id': 'chatcmpl-tool-51501bdb5f954bed96d9b122b3b0d2b3', 'type': 'tool_call'}] usage_metadata={'input_tokens': 223, 'output_tokens': 26, 'total_tokens': 249}


In [58]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"youtube_search": youtube_search, "weather_search": weather_search}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is the weather in Athens, GR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-51501bdb5f954bed96d9b122b3b0d2b3', 'type': 'function', 'function': {'name': 'weather_search', 'arguments': '{"__arg1": "Athens, GR"}'}}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 223, 'total_tokens': 249}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls'}, id='chat-7b7876f7b3314ae8ad4f9a55360cde3a', tool_calls=[{'name': 'weather_search', 'args': {'__arg1': 'Athens, GR'}, 'id': 'chatcmpl-tool-51501bdb5f954bed96d9b122b3b0d2b3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 223, 'output_tokens': 26, 'total_tokens': 249}),
 ToolMessage(content='In Athens, GR, the current weather is as follows:\nDetailed status: few clouds\nWind speed: 7.72 m/s, direction: 340°\nHumidity: 59%\nTemperature: \n  - Current: 15.75°C\n  -

In [59]:
llm_with_tools.invoke(messages)

AIMessage(content='The current weather in Athens, GR is:\n- Detailed status: few clouds\n- Wind speed: 7.72 m/s, direction: 340°\n- Humidity: 59%\n- Temperature:\n  - Current: 15.75°C\n  - High: 16.78°C\n  - Low: 15.05°C\n  - Feels like: 14.92°C\n- Rain: None\n- Heat index: None\n- Cloud cover: 20%', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 379, 'total_tokens': 506}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chat-c580681a3aca48fd8ef5d22b3d833c3e', usage_metadata={'input_tokens': 379, 'output_tokens': 127, 'total_tokens': 506})